In [37]:
from bs4 import BeautifulSoup
import requeｓts
import time
import sqlite3

In [38]:
# アクセスしたいWebサイトのURL
url = 'https://www.data.jma.go.jp/obd/stats/etrn/view/nml_sfc_d.php?prec_no=44&block_no=47662&year=2024&month=1&day=01&view=p1'

# Webサーバにリクエストを出す．レスポンスを変数に格納しておく
r = requeｓts.get(url)
r.encoding = 'utf-8'

In [39]:
html_soup = BeautifulSoup(r.text, 'html.parser') # HTMLソースをBeautifulSoupオブジェクトに変換する（プログラムで扱いやすくするため）
print(type(html_soup))

<class 'bs4.BeautifulSoup'>


In [40]:
print(html_soup.find('title').string)

気象庁｜過去の気象データ検索


In [41]:
# タグを検索し、条件に一致するものを取得
tag1 = html_soup.find_all('table', class_='data2_s')
# 結果を表示
print(tag1)

[<table class="data2_s" id="tablefix1">
<tr class="mtx"><th scope="row">要素</th><th scope="col">降水量<br/>(mm)</th><th scope="col">平均気温<br/>(℃)</th><th scope="col">最高気温<br/>(℃)</th><th scope="col">最低気温<br/>(℃)</th><th scope="col">日照時間<br/>(時間)</th><th scope="col">全天日射量<br/>(MJ/㎡)</th><th scope="col">平均雲量</th><th scope="col">降雪の深さ合計<br/>(cm)</th><th scope="col">最深積雪<br/>(cm)</th></tr>
<tr class="mtx"><th scope="row">統計期間</th><td style="text-align:center">1991～<br/>2020</td><td style="text-align:center">1991～<br/>2020</td><td style="text-align:center">1991～<br/>2020</td><td style="text-align:center">1991～<br/>2020</td><td style="text-align:center">1991～<br/>2020</td><td style="text-align:center">1991～<br/>2020</td><td style="text-align:center">1991～<br/>2020</td><td style="text-align:center">1991～<br/>2020</td><td style="text-align:center">1991～<br/>2020</td></tr>
<tr class="mtx"><th scope="row">資料年数</th><td style="text-align:center">30</td><td style="text-align:center">30</td><td style="te

In [42]:
# タグを検索し、条件に一致するものを取得
tag2 = html_soup.find_all('td', class_='data_0_0')

# テキストを取得してリストに追加するリスト内包表記を使用
data1 = [tag.string for tag in tag2]

In [43]:
data1

[]